In [6]:
import os
import pandas as pd

In [9]:
INPUT_PATH = "../Machine Learning/"
df_pcTotal = pd.read_csv(INPUT_PATH + "pca_total.csv")
patientIDs = df_pcTotal.iloc[:,0]
df_pcTotal.head()

,Unnamed: 0,PC1,PC2,PC3,PC4,PC5,PC6,PC7,PC8,PC9,...,PC145,PC146,PC147,PC148,PC149,PC150,PC151,PC152,PC153,PC154
0,t_7,0.095948,-0.028701,0.075242,0.075423,0.069380,0.004887,-0.076725,-0.130500,0.053650,...,0.004423,-0.047524,-0.014921,0.014337,0.036485,-0.035853,0.052422,0.015995,0.011706,0.000000e+00
1,t_19,0.091863,-0.125372,-0.003247,0.023471,0.097889,-0.002765,0.048405,0.043456,-0.078537,...,-0.027564,-0.055843,-0.000975,-0.029287,-0.009222,-0.069860,0.012103,0.001932,-0.045863,1.060901e-17
2,t_48,0.095019,-0.101796,-0.051884,-0.113895,-0.142038,0.001810,-0.069666,0.069405,-0.092146,...,-0.037579,-0.007205,-0.061304,0.024991,0.057245,-0.025602,0.031741,-0.018971,-0.027037,-1.388726e-17
3,m_20,0.071261,0.120673,-0.008764,-0.024208,-0.053655,-0.012222,0.068324,-0.007921,-0.084213,...,-0.338021,0.090978,0.028791,0.011494,-0.106790,0.004669,0.018704,-0.027189,0.094897,2.896854e-17
4,m_39,0.038552,-0.017600,-0.106909,0.023667,0.028861,-0.000749,-0.093223,-0.016214,-0.018393,...,-0.092402,0.102091,-0.029369,0.008334,0.025262,-0.027530,0.201096,-0.116045,-0.081053,-3.672675e-17


In [11]:
insts = [("a", "MDACC"), ("m", "MSKCC"), ("t", "Tokyo"), ("y", "Yale")]
sect = ["left", "total"]
time = ["pre", "post"]
featName=["Elongation","Flatness","LeastAxisLength","MajorAxisLength","Maximum2DDiameterColumn","Maximum2DDiameterRow","Maximum2DDiameterSlice","Maximum3DDiameter","MeshVolume","MinorAxisLength","Sphericity","SurfaceArea","SurfaceVolumeRatio","VoxelVolume"]
df_baseColNames = pd.DataFrame(columns=featName)
df_basePCA = pd.DataFrame(columns=range(1,155))
df_basePCA = df_basePCA.add_prefix("PC")

for inst in insts:
    df_lab = pd.read_excel(inst[1] + "_lab.xlsx")
    df_lab.index = df_lab["patient number"]
    folder = "csv_files_pyradiomics_" + inst[1]
    
    #add radiomics data
    if (inst[0] != "y"):
        for s in sect:
            for t in time:
                df_newR = df_baseColNames.add_prefix("shape_").add_prefix(s + '_').add_prefix(t + '_')
                for ID in patientIDs: 
                    idArray = ID.split('_')
                    if (idArray[0] == inst[0]): 
                        file = "Table_p" + idArray[1] + "_" + t + "_" + s + ".csv"
                        totalSt = os.path.join(folder, file)
                        if not os.path.exists(totalSt):
                            print("%s does not exist!" %totalSt)
                            continue
                        df_rad = pd.read_csv(totalSt)
                        colName = "p" + idArray[1] + "_" + s + "_" + t + "_label_1"
                        df_newR.loc[int(idArray[1])] = df_rad[df_rad["Feature Class"] == "shape"].iloc[:,-1].array
                df_lab = df_lab.join(df_newR)
            
    #add pca values
    for s in sect:
        df_newP = df_basePCA.add_prefix(s + '_')
        pcString = INPUT_PATH + "pca_" + s + ".csv"
        df_pc = pd.read_csv(pcString)
        for ID in patientIDs:
            idArray = ID.split('_')
            if (idArray[0] == inst[0]):
                #print(df_pc.loc[df_pc.iloc[:,0] == ID].iloc[:,1:])
                pcVals = df_pc.loc[df_pc.iloc[:,0] == ID].iloc[:,1:].values
                if (len(pcVals) == 0): 
                    print(s + ": ", ID, " - does not have any PC values!")
                    continue
                df_newP.loc[int(idArray[1])] = pcVals[0]
        df_lab = df_lab.join(df_newP)
                    
                    
    #write csv file
    df_lab = df_lab.drop(["patient number"], axis=1)
    outputS = inst[1] + "_final.xlsx"
    df_lab.to_excel(outputS)
    
    print("Finished " + inst[1])
                
                

Finished MDACC
Finished MSKCC
Finished Tokyo
Finished Yale
